**Machine Learning II - TP5 : Apprentissage par renforcement avec TF-Agents**

* **Installation des bibliothéques**

In [66]:
!pip install --upgrade --force-reinstall tensorflow==2.15.0
!pip install --upgrade --force-reinstall tf-agents[reverb]==0.16.0
!pip install --upgrade --force-reinstall numpy==1.23.5
!pip install gym

  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cac

  Using cached tf_agents-0.16.0-py3-none-any.whl.metadata (11 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gym-0.23.0-py3-none-any.whl
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pillow-11.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached protobuf-6.30.2-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached pygame-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.3 kB)
  Using cached tensorflow_pr

  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.4.38 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.38 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
moviepy 1.0.3 requires decorator<5.0,>=4.0.2, but you have decorator 5.2.1 which is incompatible.
chex 0.1.89 requires numpy>=1.24

* **Importation**

In [67]:
import tensorflow as tf
import numpy as np
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tensorflow.keras.optimizers import Adam
from tf_agents.policies import epsilon_greedy_policy
from tf_agents.utils import common
from tensorflow.keras.losses import MeanSquaredError
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.policies import random_tf_policy
from tf_agents.drivers import dynamic_step_driver

**Exercice 1 : Préparation de l'environnement**
---

**1. Création d'environnement CartPole-v0**

In [68]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

**2. Affichage des spécifications de l'environnement**

In [69]:
print("Observation specs:", tf_env.observation_spec())
print("Action specs:", tf_env.action_spec())

Observation specs: BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32))
Action specs: BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(1))


**3. Test d'environnement**

In [70]:
time_step = tf_env.reset()
while not time_step.is_last():
    action = np.random.randint(0, env.action_spec().maximum + 1)
    time_step = env.step(action)
    print(f"Observation: {time_step.observation}, Reward: {time_step.reward}, Step Type: {time_step.step_type}")

Observation: [-0.02748408  0.20482047  0.03344937 -0.24554628], Reward: 1.0, Step Type: 1
Observation: [-0.02338767  0.3994491   0.02853845 -0.52749366], Reward: 1.0, Step Type: 1
Observation: [-0.01539869  0.20393749  0.01798858 -0.22595647], Reward: 1.0, Step Type: 1
Observation: [-0.01131994  0.3987978   0.01346945 -0.51291126], Reward: 1.0, Step Type: 1
Observation: [-0.00334398  0.59372747  0.00321122 -0.80131936], Reward: 1.0, Step Type: 1
Observation: [ 0.00853057  0.39856163 -0.01281517 -0.507628  ], Reward: 1.0, Step Type: 1
Observation: [ 0.0165018   0.59386176 -0.02296773 -0.80432177], Reward: 1.0, Step Type: 1
Observation: [ 0.02837903  0.39906213 -0.03905416 -0.51895124], Reward: 1.0, Step Type: 1
Observation: [ 0.03636028  0.20451114 -0.04943319 -0.23882629], Reward: 1.0, Step Type: 1
Observation: [ 0.0404505   0.01012895 -0.05420971  0.03786337], Reward: 1.0, Step Type: 1
Observation: [ 0.04065308  0.20598468 -0.05345244 -0.2714185 ], Reward: 1.0, Step Type: 1
Observatio

**Exercice 2 : Création du réseau et de l'gent**
---

**1. Création de QNetwork**

In [71]:
q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=(100, 50)
)


**2. Création d'agent DQN**

In [72]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
train_step_counter = tf.Variable(0)

In [73]:
from tf_agents.agents.dqn import dqn_agent
agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter
)


In [74]:
agent.initialize()

**Exercice 3 : Entrainement et Evaluation**

**1. Création d'un replay buffer pour stockage des expériences**

In [75]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=100000
)

**2. Implémentation d'une boucle de collecte des données**

In [76]:
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    buffer.add_batch(traj)

**3. Implémentaton de la boucle d'entrainement**

In [81]:
num_iterations = 10000
collect_steps_per_iteration = 1
batch_size = 64

dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2
).prefetch(3)

iterator = iter(dataset)
random_policy = random_tf_policy.RandomTFPolicy(
    tf_env.time_step_spec(),
    tf_env.action_spec()
)
for _ in range(num_iterations):
    # Collect data
    for _ in range(collect_steps_per_iteration):
        collect_step(tf_env, agent.collect_policy, replay_buffer)

    # Train agent
    experience, _ = next(iterator)
    train_loss = agent.train(experience).loss

    # Log progress
    step = agent.train_step_counter.numpy()
    if step % 100 == 0:
        print(f'step = {step}: loss = {train_loss}')

step = 5100: loss = 6.723430633544922
step = 5200: loss = 10.461040496826172
step = 5300: loss = 13.368563652038574
step = 5400: loss = 9.804285049438477
step = 5500: loss = 28.584152221679688
step = 5600: loss = 29.89196014404297
step = 5700: loss = 9.770882606506348
step = 5800: loss = 10.577302932739258
step = 5900: loss = 7.501821517944336
step = 6000: loss = 11.675416946411133
step = 6100: loss = 41.90787124633789
step = 6200: loss = 27.883014678955078
step = 6300: loss = 30.549579620361328
step = 6400: loss = 21.605113983154297
step = 6500: loss = 13.092782974243164
step = 6600: loss = 149.81544494628906
step = 6700: loss = 5.6103644371032715
step = 6800: loss = 5.5682783126831055
step = 6900: loss = 5.760518550872803
step = 7000: loss = 41.35927200317383
step = 7100: loss = 51.664119720458984
step = 7200: loss = 50.924476623535156
step = 7300: loss = 21.94211196899414
step = 7400: loss = 28.024097442626953
step = 7500: loss = 70.06224822998047
step = 7600: loss = 67.120750427246

**4. Evaluation des performances de l'agent**

In [82]:
def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

print('Untrained policy:')
print(compute_avg_return(tf_env, agent.policy))


print('Trained policy:')
print(compute_avg_return(tf_env, agent.policy))

Untrained policy:
146.5
Trained policy:
163.7
